In [ ]:
import os
import sys
os.environ['GLEAMS_HOME'] = os.path.join(os.environ['HOME'],
                                         'Projects/gleams')
# Make sure all code is in the PATH.
sys.path.append(
    os.path.normpath(os.path.join(os.environ['GLEAMS_HOME'], 'src')))

In [ ]:
import collections
import math
from typing import List

import joblib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import pyteomics
import seaborn as sns

In [ ]:
import logging
logger = logging.getLogger('gleams')
logger.setLevel(logging.DEBUG)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from gleams.dag import dag

from gleams import config
from gleams.ms_io import ms_io
from gleams.cluster import cluster

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

## Clustering

Cluster the spectra (using the previously determined optimal clustering hyperparameters).

In [ ]:
config.eps = 0.02
config.min_samples = 2
os.remove(os.path.join(os.environ['GLEAMS_HOME'], 'data', 'ann',
                       f'clusters_{config.massivekb_task_id}.npy'))
cluster.cluster(os.path.join(os.environ['GLEAMS_HOME'], 'data', 'ann',
                             f'dist_{config.massivekb_task_id}.npz'))

In [ ]:
clusters = pd.merge(
    pd.read_parquet(
        os.path.join(os.environ['GLEAMS_HOME'], 'data', 'embed',
                     f'embed_{config.massivekb_task_id}.parquet')),
    pd.read_parquet(
        os.path.join(os.environ['GLEAMS_HOME'], 'data', 'metadata',
                     f'metadata_{config.massivekb_task_id}.parquet')),
    'left', ['dataset', 'filename', 'scan'])
# Don't disambiguate between I/L.
clusters['sequence'] = clusters['sequence'].str.replace('I', 'L')
clusters['cluster'] = np.load(os.path.join(
    os.environ['GLEAMS_HOME'], 'data', 'ann',
    f'clusters_{config.massivekb_task_id}.npy'))

In [ ]:
num_id_massive = len(clusters['sequence'].dropna())
print(f'Number of initial spectrum identifications: {num_id_massive:,} '
      f'({(num_id_massive / len(clusters)):.2%})')

## Spectrum export

Export the unidentified spectra that are part of valid clusters to MGF files used for reidentification. Spectra belonging to the noise cluster are not exported.

We hypothesize the clustered spectra correspond to higher quality spectra that are repeatedly measured.

In [ ]:
reident_peak_dir = os.path.join(os.environ['GLEAMS_HOME'], 'data', 'ann',
                                'peak')
os.makedirs(reident_peak_dir, exist_ok=True)
existing_peak_files = set([os.path.splitext(filename)[0]
                           for filename in os.listdir(reident_peak_dir)
                           if filename.endswith('.mgf')])

In [ ]:
clusters_non_ident = (clusters[clusters['cluster'] != -1].groupby('cluster')
                      ['sequence'].apply(lambda sequences:
                                         all(pd.isnull(sequences))))
clusters_non_ident = clusters_non_ident[clusters_non_ident].index
dataset_filename_scans = (clusters[clusters['cluster']
                                   .isin(clusters_non_ident)]
                          .groupby(['dataset', 'filename'])
                          ['scan'].apply(list)
                          .reset_index())

In [ ]:
num_unid_export = dataset_filename_scans['scan'].apply(len).sum()
print(f'Number of clustered, unidentified spectra: {num_unid_export:,} '
      f'({(num_unid_export / len(clusters)):.2%})')

In [ ]:
def get_spectra_from_file(dataset: str, filename: str, scans: List[int]):
    logger.debug('Process file %s/%s', dataset, filename)
    return list(ms_io.get_spectra(os.path.join(
        os.environ['GLEAMS_HOME'], 'data', 'peak', dataset, filename)))

In [ ]:
logger.info('Read and export clustered, unidentified spectra to MGF files '
            'in directory %s', reident_peak_dir)
datasets_to_export = dataset_filename_scans[
    ~dataset_filename_scans['dataset'].isin(existing_peak_files)]
dataset_total = datasets_to_export['dataset'].nunique()
for i, (dataset, filename_scans) in enumerate(
        datasets_to_export.groupby('dataset'), 1):
    logger.debug('Process dataset %s [%3d/%3d]', dataset, i, dataset_total)
    spectra_dicts = []
    for filename, spectra in zip(
            filename_scans['filename'],
            joblib.Parallel(n_jobs=-1, backend='multiprocessing')(
                joblib.delayed(get_spectra_from_file)
                (dataset, filename, scans)
                for filename, scans in zip(filename_scans['filename'],
                                           filename_scans['scan']))):
        spectra_dicts.extend([
            {'m/z array': spectrum.mz,
            'intensity array': spectrum.intensity,
            'params': {
                'TITLE': f'mzspec:{dataset}:' \
                    f'{os.path.splitext(filename)[0]}:' \
                    f'scan:{spectrum.identifier}',
                'RTINSECONDS': spectrum.retention_time,
                'PEPMASS': spectrum.precursor_mz,
                'CHARGE': f'{spectrum.precursor_charge}+'}}
            for spectrum in spectra])
    with open(os.path.join(reident_peak_dir, f'{dataset}.mgf'), 'w') as f:
        pyteomics.mgf.write(spectra_dicts, f)

## Incorporate ANN-SoLo identifications

Include the identifications from ANN-SoLo processing of the previously exported unidentified spectra.

In [ ]:
def read_mztab_psms(filename: str) -> pd.DataFrame:
    """
    Read PSMs from the given mzTab file.
    
    Parameters
    ----------
    filename: str
        The mzTab file name from which to read the PSMs.
    
    Returns
    -------
    pd.DataFrame
        A data frame containing the PSM information from the mzTab file.
    """
    # Skip the header lines.
    skiplines = 0
    with open(filename) as f_in:
        line = next(f_in)
        while line.split('\t', 1)[0] != 'PSH':
            line = next(f_in)
            skiplines += 1
    return pd.read_csv(filename, sep='\t', header=skiplines,
                       index_col='PSM_ID').drop(columns='PSH')

In [ ]:
reident_id_dir = os.path.join(os.environ['GLEAMS_HOME'], 'data', 'ann', 'id')
psms = pd.concat([read_mztab_psms(os.path.join(reident_id_dir, filename))
                  for filename in os.listdir(reident_id_dir)
                  if os.path.splitext(filename)[1] == '.mztab'])
psms['sequence'] = psms['sequence'].str.replace('I', 'L')

In [ ]:
num_id_ann_solo = len(psms)
print(f'Number of ANN-SoLo spectrum identifications: {num_id_ann_solo:,} '
      f'({(num_id_ann_solo / len(clusters)):.2%}; '
      f'{(num_id_ann_solo / num_unid_export):.2%} of previously unidentified '
      f'spectra)')

In [ ]:
clusters = clusters.set_index(
    'mzspec:' + clusters['dataset'] + ':' +
    clusters['filename'].str.rsplit('.', 1, True)[0] + ':scan:' +
    clusters['scan'].astype(str))

In [ ]:
clusters.loc[psms.index, 'sequence'] = psms['sequence']
clusters['ann_solo'] = False
clusters.loc[psms.index, 'ann_solo'] = True

### Mass difference histogram

In [ ]:
def get_mass_groups(psms, tol_mass, tol_mode, min_group_size=None):
    psms_remaining = psms.sort_values('search_engine_score[1]',
                                      ascending=False)
    psms_remaining['mass_diff'] = ((psms_remaining['exp_mass_to_charge'] -
                                    psms_remaining['calc_mass_to_charge']) *
                                   psms_remaining['charge'])

    # Start with the highest ranked SSM.
    mass_groups = []
    while psms_remaining.size > 0:
        # Find all remaining PSMs within the mass difference window.
        mass_diff = psms_remaining['mass_diff'].iat[0]
        if (tol_mass is None or tol_mode not in ('Da', 'ppm') or
                min_group_size is None):
            mask = np.full(len(psms_remaining), True, dtype=bool)
        elif tol_mode == 'Da':
            mask = (np.fabs(psms_remaining['mass_diff'] - mass_diff) <=
                    tol_mass)
        elif tol_mode == 'ppm':
            mask = (np.fabs(psms_remaining['mass_diff'] - mass_diffs) /
                    psms_remaining['exp_mass_to_charge'] * 10 ** 6
                    <= tol_mass)
        mass_groups.append(psms_remaining[mask])
        # Exclude the selected PSMs from further selections.
        psms_remaining = psms_remaining[~mask]

    mass_group_stats = []
    for mass_group in mass_groups:
        mass_group_stats.append((mass_group['mass_diff'].median(),
                                 mass_group['mass_diff'].mean(),
                                 len(mass_group)))
    mass_group_stats = pd.DataFrame.from_records(
        mass_group_stats, columns=['mass_diff_median', 'mass_diff_mean',
                                   'num_psms'])
    return mass_group_stats

In [ ]:
tol_mass, tol_mode, min_group_size = 0.1, 'Da', 20
mass_groups = get_mass_groups(psms, tol_mass, tol_mode, min_group_size)

In [ ]:
mass_groups.sort_values('num_psms', ascending=False).head(20)

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width * 1.5, height / 1.5))

# Exclude unmodified PSMs.
mask = mass_groups['mass_diff_median'].abs() > tol_mass
ax.bar(mass_groups[mask]['mass_diff_median'], mass_groups[mask]['num_psms'],
       width=0.4, color='black')

# Annotate the most frequent modifications.
modifications = [('oxidation', 0, 14000),
                 ('first isotopic peak', -15, 12000),
                 ('SILAC\nlabel', 0, 10000),
                 (None, None, None),
                 (None, None, None),
                 ('dehydration', 0, 5000),
                 (None, None, None),
                 (None, None, None),
                 (None, None, None),
                 ('Lys→Arg', 0, 5000),
                 ('phosphorylation', 0, 5000),
                 (None, None, None),
                 (None, None, None),
                 (None, None, None),
                 (None, None, None),
                 ('3 protons to iron', 0, 3500)]
for (annot, x, y), mass_group in zip(modifications, mass_groups.sort_values(
        'num_psms', ascending=False)[1:].itertuples()):
    if annot is not None:
        ax.annotate(annot,
                    (mass_group.mass_diff_median, mass_group.num_psms + 50),
                    (mass_group.mass_diff_median + x, y),
                    arrowprops={'arrowstyle': '<-', 'linewidth': 1},
                    ha='center')

ax.set_xlim((-50, 100))

ax.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))

sns.despine(ax=ax)

ax.set_xlabel('Precursor mass difference (Da)')
ax.set_ylabel(f'Number of PSMs (FDR=1%)')

# plt.savefig('mass_diff.pdf', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## Propagate identifications

We can propagate identifications within clusters by assigning unmodified spectra the same peptide sequence as the majority of identified spectra in the same cluster.

We check both identification propagations based on the initial identifications and additional identification propagations based on the new ANN-SoLo identifications.

In [ ]:
def _get_cluster_num_id_propagation(sequences):
    num_ids = pd.notnull(sequences).sum()
    return len(sequences) - num_ids if num_ids > 0 else 0

In [ ]:
def _get_num_id_propagation(clusters):
    return clusters.groupby('cluster')['sequence'].apply(
        _get_cluster_num_id_propagation).sum()

num_id_prop_initial = _get_num_id_propagation(
    clusters[(clusters['cluster'] != -1) & ~clusters['ann_solo']])
num_id_prop_ann_solo = _get_num_id_propagation(
    clusters[clusters['cluster'] != -1]) - num_id_prop_initial

In [ ]:
print(f'Number of propagated initial spectrum identifications: '
      f'{num_id_prop_initial:,} '
      f'({(num_id_prop_initial / len(clusters)):.2%})')
print(f'Number of propagated ANN-SoLo spectrum identifications: '
      f'{num_id_prop_ann_solo:,} '
      f'({(num_id_prop_ann_solo / len(clusters)):.2%})')

In [ ]:
width = 7
height = width / 1.618
fig, ax = plt.subplots(figsize=(width / 2, height)

num_ids = pd.DataFrame([[num_id_massive, num_id_prop_initial,
                         num_id_ann_solo, num_id_prop_ann_solo,
                         len(clusters[clusters['cluster'] != -1].dropna()),
                         len(clusters[clusters['cluster'] == -1])]],
                       columns=['original identifications',
                                'original identification propagation',
                                'ANN-SoLo identifications',
                                'ANN-SoLo identification propagation',
                                'clustered unidentified',
                                'unclustered'])

barlist = num_ids.plot.bar(ax=ax, stacked=True, color=[
    *sns.color_palette(n_colors=len(num_ids.columns) - 1), 'lightgray'])

ax.set_xticks([])
ax.set_ylabel('Number of identified spectra')

h, l = ax.get_legend_handles_labels()
ax.legend(h[::-1], l[::-1], loc='center left', bbox_to_anchor=(1, 0.5))

ax.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))

sns.despine(ax=ax)

# plt.savefig('id_propagation.pdf', dpi=300, bbox_inches='tight')
plt.show()
plt.close()